# Podstawy programowania w analizie danych

## Tomasz Rodak

2017/2018, semestr letni

Wykład VIII

# Klasy
## Przegląd metod specjalnych, cz. 2
## Obiekty zachowujące się jak liczby

## `__add__(self, y)`

Ta metoda jest wywoływana przez operator `+`.

Jeśli `x` i `y` są obiektami i `x` posiada metodę `__add__()`, to wyrażenie
```python
x + y
```
jest równoważne z
```python
x.__add__(y)
```

## `GodzinaZegarowa` -- wersja 1.0

Piszemy klasę, która tworzy obiekty odpowiadające godzinie zegarowej.

Opiszmy sposób jej użycia w testach.

In [89]:
g = GodzinaZegarowa()

assert str(g) == '00:00:00'
assert repr(g) == 'GodzinaZegarowa(h=0, m=0, s=0)'

g = GodzinaZegarowa(h=9, m=32, s=5)

assert str(g) == '09:32:05'
assert repr(g) == 'GodzinaZegarowa(h=9, m=32, s=5)'

okres = dict(h=10, m=30, s=40)
g += okres

assert str(g) == '20:02:45'

okres = dict(h=5)
g += okres

assert str(g) == '01:02:45'

Dodawanie okresu do godziny zegarowej możemy przeprowadzić na sekundach.

Potrzebujemy zatem konwertera od godziny zegarowej do sekund i z powrotem.

Znów napiszmy testy.

In [87]:
godzina_zegarowa = dict(h=13, m=46, s=40)

assert na_godzinę_zegarową(na_sekundy(**godzina_zegarowa)) == godzina_zegarowa

# Milion sekund to
# 11 dni, 13 godzin, 46 minut i 40 sekund.
assert na_sekundy(**na_godzinę_zegarową(10**6)) == 13*3600 + 46*60 + 40

`na_sekundy(h, m, s)` przelicza dowolny okres wyrażony w godzinach, minutach i sekundach na sekundy. Wszystkie parametry domyślnie mają wartość `0`.

In [104]:
def na_sekundy(h=0, m=0, s=0):
#     h, m, s = okres.get('h', 0), okres.get('m', 0), okres.get('s', 0)
    return h*3600 + m*60 +s

`na_godzinę_zegarową(s)` wyznacza godziną jaką będzie wskazywał zegar po `s` sekundach od godziny `00:00:00`. Wartość godziny przekazywana jest w słowniku o kluczach `'h'`, `'m'` i `'s'`.

In [108]:
def na_godzinę_zegarową(s):
    h, m = divmod(s, 3600)
    m, s = divmod(m, 60)
    return dict(h=h % 24, m=m, s=s)

In [68]:
class GodzinaZegarowa:
    
    def __init__(self, h=0, m=0, s=0):
        if not (0 <= h < 24 and 0 <= m < 60 and 0 <= s < 60):
            raise ValueError('Nieprawidłowy format godziny zegarowej.')

        self.h, self.m, self.s = h, m, s
    
    def __repr__(self):
        return 'GodzinaZegarowa(h={}, m={}, s={})'.format(self.h, self.m, self.s)
    
    def __str__(self):
        return '{:0>2}:{:0>2}:{:0>2}'.format(self.h, self.m, self.s)
    
    def __add__(self, okres):
        godzina_zegarowa = dict(h=self.h, m=self.m, s=self.s)
        sekundy = na_sekundy(**godzina_zegarowa) + na_sekundy(**okres)
        return GodzinaZegarowa(**na_godzinę_zegarową(sekundy))

## `__sub__(self, y)`

Ta metoda jest wywoływana przez operator `-`.

Jeśli `x` i `y` są obiektami i `x` posiada metodę `__sub__()`, to wyrażenie
```python
x - y
```
jest równoważne z
```python
x.__sub__(y)
```

## `GodzinaZegarowa` -- wersja 2.0

Dopisujemy możliwość odejmowania okresu od godziny zegarowej.

Oto sposób użycia w testach.

In [111]:
g = GodzinaZegarowa()

g -= dict(s=100)
assert str(g) == '23:58:20'

g -= dict(m=24 * 60)
assert str(g) == '23:58:20'

g = GodzinaZegarowa(h=10, m=12, s=27)
g -= dict(s=10**8)
g += dict(s=10**8)
assert str(g) == '10:12:27'

g -= dict(h=2, m=15, s=3)

assert str(g) == '07:57:24'

Okazuje się, że w samym przeliczaniu nic nie musimy zmieniać. Funkcja `na_godzinę_zegarową(s)` zwraca dla ujemnego `s` wynik taki, jakiego właśnie potrzebujemy.

In [112]:
na_godzinę_zegarową(-100)

{'h': 23, 'm': 58, 's': 20}

Teraz wystarczy do klasy dopisać metodę `__sub__()`.

In [105]:
class GodzinaZegarowa:
    
    def __init__(self, h=0, m=0, s=0):
        if not (0 <= h < 24 and 0 <= m < 60 and 0 <= s < 60):
            raise ValueError('Nieprawidłowy format godziny zegarowej.')
        
        self._gz = dict(h=h, m=m, s=s)
        self.h, self.m, self.s = h, m, s
    
    def __repr__(self):
        return 'GodzinaZegarowa(h={h}, m={m}, s={s})'.format(**self._gz)
    
    def __str__(self):
        return '{h:0>2}:{m:0>2}:{s:0>2}'.format(**self._gz)
    
    def __add__(self, okres):
        sekundy = na_sekundy(**self._gz) + na_sekundy(**okres)
        return GodzinaZegarowa(**na_godzinę_zegarową(sekundy))
    
    def __sub__(self, okres):
        sekundy = na_sekundy(**self._gz) - na_sekundy(**okres)
        return GodzinaZegarowa(**na_godzinę_zegarową(sekundy))

## Metody działające podobnie do `__add__()`

* `__add__(self, y)` -- dodawanie `+`;
* `__sub__(self, y)` -- odejmowanie `-`;
* `__mul__(self, y)` -- mnożenie `*`;
* `__matmul__(self, y)` -- mnożenie macierzowe `@`;
* `__truediv__(self, y)` -- dzielenie zmiennoprzecinkowe `/`;
* `__floordiv__(self, y)` -- dzielnie podłogowe `//`;
* `__mod__(self, y)` -- reszta z dzielenia `%`;
* `__divmod__(self, y)` -- `divmod()`;
* `__pow__(self, y[, modulo])` -- potęga `**`;
* `__lshift__(self, y)` -- przesunięcie bitowe w lewo o `y` pozycji `<<`;
* `__rshift__(self, y)` -- przesunięcie bitowe w prawo o `y` pozycji `>>`;
* `__and__(self, y)` -- koniunkcja bitowa `&`;
* `__xor__(self, y)` -- bitowa alternatywa rozłączna `^`;
* `__or__(self, y)` -- alternatywa bitowa `|`.

## `Wektor`

Obiekty klasy `Wektor` mają być wektorami. Nasze wektory chcemy do siebie dodawać i mnożyć przez liczbę.

Oto testy

In [5]:
v = Wektor(2, 3)

assert str(v) == 'Wektor(2, 3)'
assert repr(v) == 'Wektor(2, 3)'
assert str(-v) == 'Wektor(-2, -3)'

w = Wektor(-5, 7)

assert str(v + w) == 'Wektor(-3, 10)'
assert str(w - 2*v) == 'Wektor(-9, 1)'
assert str(4 * w) == 'Wektor(-20, 28)'

z = Wektor(1, 2, 3)

try:
    w + z
except ValueError:
    pass
else:
    AssertionError('Konflikt wymiarów.')

* Ponieważ `repr()` i `str()` zwracają to samo, więc piszemy tylko `__repr__()`.
* Operator unarny `-self` wywołuje metodę specjalną `__neg__(self)`.

In [8]:
class Wektor:
    
    def __init__(self, *wsp):
        self.wsp = wsp
    
    def __repr__(self):
        return 'Wektor({})'.format(', '.join(str(x) for x in self.wsp))
    
    def __add__(self, w):
        if len(self.wsp) != len(w.wsp):
            raise ValueError('Konflikt wymiarów.')
        
        wynik_wsp = tuple(x + y for x, y in zip(self.wsp, w.wsp))
        return Wektor(*wynik_wsp)
    
    def __mul__(self, skalar):
        wynik_wsp = tuple(skalar * x for x in self.wsp)
        return Wektor(*wynik_wsp)        
    
    def __neg__(self):
        return -1 * self
    
    def __sub__(self, w):
        return self + -w

<div class="alert alert-block alert-danger"><b>Błąd!</b> Testy nie przechodzą, gdyż skalar występuje po lewej stronie wektora.</div>

## `__rmul__(self, x)`

Jeśli `x`, `y` są obiektami i `x` nie ma metody `__mul__()` lub `x.__mul__(y)` jest niewykonalne, to wyrażenie
```python
x * y
```
wywołuje
```python
y.__rmul__(x)
```

## `Wektor` -- wersja 1.0

Wszystko co teraz trzeba zrobi, to zamienić `__mul__()` na `__rmul__()`.

In [10]:
class Wektor:
    
    def __init__(self, *wsp):
        self.wsp = wsp
    
    def __repr__(self):
        return 'Wektor({})'.format(', '.join(str(x) for x in self.wsp))
    
    def __add__(self, w):
        if len(self.wsp) != len(w.wsp):
            raise ValueError('Konflikt wymiarów.')
        
        wynik_wsp = tuple(x + y for x, y in zip(self.wsp, w.wsp))
        return Wektor(*wynik_wsp)
    
    def __rmul__(self, skalar):
        wynik_wsp = tuple(skalar * x for x in self.wsp)
        return Wektor(*wynik_wsp)
    
    def __neg__(self):
        return -1 * self
    
    def __sub__(self, w):
        return self + -w

Zwróć uwagę, że teraz skalar **musi** być pierwszym czynnikiem iloczynu.

In [175]:
w = Wektor(2.4, 6)

3 * w

Wektor(7.199999999999999, 18)

In [176]:
w * 3

TypeError: unsupported operand type(s) for *: 'Wektor' and 'int'

## Metody działające podobnie do `__rmul__()`

* `__radd__(self, x)`
* `__rsub__(self, x)`
* `__rmul__(self, x)`
* `__rmatmul__(self, x)`
* `__rtruediv__(self, x)`
* `__rfloordiv__(self, x)`
* `__rmod__(self, x)¶`
* `__rdivmod__(self, x)`
* `__rpow__(self, x)`
* `__rlshift__(self, x)`
* `__rrshift__(self, x)`
* `__rand__(self, x)`
* `__rxor__(self, x)`
* `__ror__(self, x)`

## Przegląd metod specjalnych cz. 3
## Porównywanie obiektów

## `__eq__(self, y)`, `__ne__(self, y)`

* Te metody wywoływane są przez operatory porównywania `==` i `!=`.
* Jeśli nie zwracają bezpośrednio `True/False`, to na wartości zwracanej wywoływane jest `bool()`.
* `__ne__()` wywołuje `__eq__()` i zwraca zaprzeczenie.

## `GodzinaZegarowa` -- wersja 3.0

Dopisujemy możliwość porównywania, czy dwie godziny zegarowe mają tę samą wartość.

Oto sposób użycia w testach.

In [207]:
g = GodzinaZegarowa(h=12, m=30, s=17)
g_prim = GodzinaZegarowa(h=12, m=30, s=17)

assert g == g_prim

okres = dict(m=100, s=200)

g += okres

assert g != g_prim

g_prim += okres

assert g == g_prim

In [206]:
class GodzinaZegarowa:
    
    def __init__(self, h=0, m=0, s=0):
        if not (0 <= h < 24 and 0 <= m < 60 and 0 <= s < 60):
            raise ValueError('Nieprawidłowy format godziny zegarowej.')
        
        self._gz = dict(h=h, m=m, s=s)
        self.h, self.m, self.s = h, m, s
    
    def __repr__(self):
        return 'GodzinaZegarowa(h={h}, m={m}, s={s})'.format(**self._gz)
    
    def __str__(self):
        return '{h:0>2}:{m:0>2}:{s:0>2}'.format(**self._gz)
    
    def __add__(self, okres):
        sekundy = na_sekundy(**self._gz) + na_sekundy(**okres)
        return GodzinaZegarowa(**na_godzinę_zegarową(sekundy))
    
    def __sub__(self, okres):
        sekundy = na_sekundy(**self._gz) - na_sekundy(**okres)
        return GodzinaZegarowa(**na_godzinę_zegarową(sekundy))
    
    def __eq__(self, inna_godzina):
        return self._gz == inna_godzina._gz

## Metody porównujące

* `__lt__(self, y)` -- `<`
* `__le__(self, y)` -- `<=`
* `__eq__(self, y)` -- `==`
* `__ne__(self, y)` -- `!=`
* `__gt__(self, y)` -- `>`
* `__ge__(self, y)` -- `>=`

## `GodzinaZegarowa` -- wersja 3.1

Dopisujemy możliwość porównywania, która godzina jest wcześniejsza.

Testy:

In [240]:
g = GodzinaZegarowa(h=12, m=30, s=17)
g_prim = GodzinaZegarowa(h=12, m=30, s=17)

assert g == g_prim

okres = dict(s=1000)

assert g - okres < g_prim
assert g + okres > g_prim
assert g - okres <= g_prim - okres
assert g - okres == g_prim - okres
assert g - okres >= g_prim - okres
assert g + dict(s=24*3600 + 987) > g_prim

In [236]:
class GodzinaZegarowa:
    
    def __init__(self, h=0, m=0, s=0):
        if not (0 <= h < 24 and 0 <= m < 60 and 0 <= s < 60):
            raise ValueError('Nieprawidłowy format godziny zegarowej.')
        
        self._gz = dict(h=h, m=m, s=s)
        self.h, self.m, self.s = h, m, s
    
    def __repr__(self):
        return 'GodzinaZegarowa(h={h}, m={m}, s={s})'.format(**self._gz)
    
    def __str__(self):
        return '{h:0>2}:{m:0>2}:{s:0>2}'.format(**self._gz)
    
    def __add__(self, okres):
        sekundy = na_sekundy(**self._gz) + na_sekundy(**okres)
        return GodzinaZegarowa(**na_godzinę_zegarową(sekundy))
    
    def __sub__(self, okres):
        sekundy = na_sekundy(**self._gz) - na_sekundy(**okres)
        return GodzinaZegarowa(**na_godzinę_zegarową(sekundy))
    
    def __eq__(self, inna_godzina):
        return self._gz == inna_godzina._gz
    
    def __lt__(self, inna_godzina):
        return na_sekundy(**self._gz) < na_sekundy(**inna_godzina._gz)
    
    def __le__(self, inna_godzina):
        return na_sekundy(**self._gz) <= na_sekundy(**inna_godzina._gz)

## `Wektor` -- wersja 2.0

Dopisujemy możliwość porównywania, czy dwa wektory mają tę sama wartość.

Testy

In [6]:
w = Wektor(3, 4)
v = Wektor(6, 8)

assert 2 * w == v

z = Wektor(4, 3)

assert z != w

y = Wektor(3, 4, 5)

assert w != y

In [4]:
class Wektor:
    
    def __init__(self, *wsp):
        self.wsp = wsp
    
    def __repr__(self):
        return 'Wektor({})'.format(', '.join(str(x) for x in self.wsp))
    
    def __add__(self, w):
        if len(self.wsp) != len(w.wsp):
            raise ValueError('Konflikt wymiarów.')
        
        wynik_wsp = tuple(x + y for x, y in zip(self.wsp, w.wsp))
        return Wektor(*wynik_wsp)
    
    def __rmul__(self, skalar):
        wynik_wsp = tuple(skalar * x for x in self.wsp)
        return Wektor(*wynik_wsp)
    
    def __neg__(self):
        return -1 * self
    
    def __sub__(self, w):
        return self + -w
    
    def __eq__(self, w):
        if len(self.wsp) != len(w.wsp):
            return False
        
        return all(x == y for x, y in zip(self.wsp, w.wsp))

## Oblicz rząd macierzy

<br>
```
 2  8   3 -4
 1  4   1 -2
 5  20  0 -10
-3 -12 -2  6
```

In [9]:
w1 = Wektor(2, 1, 5, -3)
w2 = Wektor(8, 4, 20, -12)
w3 = Wektor(3, 1, 0, -2)
w4 = Wektor(-4, -2, -10, 6)

print(w1, w2, w3, w4, sep='\n')

Wektor(2, 1, 5, -3)
Wektor(8, 4, 20, -12)
Wektor(3, 1, 0, -2)
Wektor(-4, -2, -10, 6)


In [10]:
from fractions import Fraction as F

w1, w2, w3, w4 = w1, w2 - 4*w1, w3 - F(3, 2) * w1, w4 + 2*w1

print(w1, w2, w3, w4, sep='\n')

Wektor(2, 1, 5, -3)
Wektor(0, 0, 0, 0)
Wektor(0, -1/2, -15/2, 5/2)
Wektor(0, 0, 0, 0)


In [11]:
# Zmieniamy kolejność ustawiając postać trójkątną.

w1, w2, w3, w4 = w1, w3, w2, w4

print(w1, w2, w3, w4, sep='\n')

Wektor(2, 1, 5, -3)
Wektor(0, -1/2, -15/2, 5/2)
Wektor(0, 0, 0, 0)
Wektor(0, 0, 0, 0)


In [12]:
# Mamy postać trójkątną. Rząd jest równy 2.

print(w1, w2, w3, w4, sep='\n')

Wektor(2, 1, 5, -3)
Wektor(0, -1/2, -15/2, 5/2)
Wektor(0, 0, 0, 0)
Wektor(0, 0, 0, 0)
